# Data exploration - combining data sources

This notebook looks at the latest COVID-19 cases and deaths information available through https://coronavirus.data.gov.uk/ and combines it with geographical and deprivation index information.

In [ ]:
import pandas as pd
import requests
from io import StringIO
import numpy as np
def download_latest_csvs(url):
    reqCSV=requests.get(url).text
    csvDF=pd.read_csv(StringIO(reqCSV))
    return csvDF
import matplotlib.pyplot as plt

## Obtaining and cleaning data

Deprivation by area data is just a fixed file as it was published in 2019

In [ ]:
depDF=pd.read_csv("../data/deprivation_index_by_area.csv")

In [ ]:
depDF['http://opendatacommunities.org/def/ontology/geography/refArea'][0]

In [ ]:
depDF.columns.tolist()

In [ ]:
depDF.columns=['AreaCode',
 'Reference area',
 'a. Index of Multiple Deprivation (IMD)',
 'b. Income Deprivation Domain',
 'c. Employment Deprivation Domain',
 'd. Education, Skills and Training Domain',
 'e. Health Deprivation and Disability Domain',
 'f. Crime Domain',
 'g. Barriers to Housing and Services Domain',
 'h. Living Environment Deprivation Domain',
 'i. Income Deprivation Affecting Children Index (IDACI)',
 'j. Income Deprivation Affecting Older People Index (IDAOPI)']

In [ ]:
depDF['AreaCode']

Get the area code from these URLs:

In [ ]:
areaCodeList=depDF['AreaCode'].str.rsplit(pat='/',n=1).tolist()

In [ ]:
areaCodeList=np.array(areaCodeList)

In [ ]:
areaCodeList=areaCodeList[:,-1]

In [ ]:
depDF['AreaCode']=areaCodeList

In [ ]:
depDF

In [ ]:
depDF.columns

In [ ]:
ddf=depDF[['AreaCode','a. Index of Multiple Deprivation (IMD)']]

In [ ]:
ddf.columns=['Area code','IMD']

In [ ]:
ddf

Request the latest available datasets from data.gov.uk

In [ ]:
cases_url = "https://coronavirus.data.gov.uk/downloads/csv/coronavirus-cases_latest.csv"
deaths_url = "https://coronavirus.data.gov.uk/downloads/csv/coronavirus-deaths_latest.csv"
casesDF= download_latest_csvs(cases_url)
deathsDF= download_latest_csvs(deaths_url)

In [ ]:
deathsDF

Deaths information is given at the Nation level unlike the cases information, so is less useful for us here if we want to merge it with deprivation information for English local authorities

Work with cases information:

In [ ]:
casesDF.sort_values(by='Specimen date')

Merge the COVID-19 cases statistics with the IMD information when the area code matches:

In [ ]:
cases_ddf=pd.merge(casesDF, ddf, how='inner', on='Area code')

In [ ]:
cases_ddf

Request the latest available date from the dataframe:

In [ ]:
cases_areas_latest=cases_ddf[(cases_ddf['Specimen date']==cases_ddf['Specimen date'].max()) & (cases_ddf['Area type'].str.match('Lower tier local authority'))]
cases_areas_latest

In [ ]:
calIMD=cases_areas_latest.sort_values(by='IMD')

In [ ]:
calIMD

Remove empty columns:

In [ ]:
calIMD=calIMD.dropna(axis=1,how='all')

## Starting to visualise the data

Try a basic scatter plot of cumulative cases vs IMD

In [ ]:
calIMD.plot.scatter("IMD",'Cumulative lab-confirmed cases')

A 2D histogram is better for this purpose, so use matplotlib.pyplot and try a few styles!

In [ ]:
plt.hist2d(calIMD['IMD'],calIMD['Cumulative lab-confirmed cases'])

In [ ]:
npIMD=calIMD['IMD'].to_numpy()

In [ ]:
npCases=calIMD['Cumulative lab-confirmed cases'].to_numpy()

In [ ]:
plt.figure(figsize=(8,6))
hexbin=plt.hexbin(npIMD,npCases,gridsize=10,cmap='jet',mincnt=1)
plt.colorbar(hexbin)

The below uses matplotlib.pyplot directly rather than going through pandas, offering more direct control of the parameters

In [ ]:
plt.figure(figsize=(8,6))
histTwoD=plt.hist2d(npIMD,npCases,cmap='jet')

The `jet` colour map is (notoriously) contentious - see this [colour map advice](https://www.kennethmoreland.com/color-advice/) for scientific plotting. Try with `viridis`, and add a colour bar for the number of bin entries

In [ ]:
fig, ax = plt.subplots()
h = ax.hist2d(npIMD, npCases, bins=20,cmin=1,cmap='viridis')
plt.colorbar(h[3], ax=ax)

Can then go on to add titles, axis labels, customise bin numbers...

In [ ]:
import geopandas as gpd

In [ ]:
geoURL="https://opendata.arcgis.com/datasets/a8531598f29f44e7ad455abb6bf59c60_0.geojson"

Load in a geopandas DataFrame directly from a URL pointing at a geojson:

In [ ]:
#mapdf=gpd.read_file(geoURL)

...or one saved locally

In [ ]:
mapdf=gpd.read_file('../data/geofiles/geofile.shp')

In [ ]:
mapdf

If you've loaded the local file, you don't need to trim the file as below

In [ ]:
#mapdf=mapdf[['lad19cd','lad19nm','geometry']]
#mapdf=.columns=['Area code','Area name','geometry']

Call geopandas.plot() to see what we've loaded

In [ ]:
mapdf.plot()

Merge the map information with the cases information retrieved previously

In [ ]:
merged=pd.merge(mapdf,calIMD,on='Area code',how='inner')

In [ ]:
merged

Select a subset of columns:

In [ ]:
merged=merged[['geometry','Area name','Specimen date','Cumulative lab-confirmed cases','Cumulative lab-confirmed cases rate','IMD']]

Rename the columns for convenience

In [ ]:
merged.columns=['geometry', 'Area', 'Date',
       'Cases', 'RatePer100K',
       'IMD']

### Start overlaying England local authority regions onto a UK map with case information

In [ ]:
crsdf = merged.to_crs(epsg=3857)

In [ ]:
import contextily as ctx

In [ ]:
ax = crsdf.plot(column='Cases',cmap='Blues',figsize=(10, 10), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)

I don't find this plot very clear so haven't included it in the main code, but it's more here as a proof of concept

An alternative view using matplotlib:

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 8))
merged.plot(column='Cases', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')
sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=1,vmax=3500))
sm._A = []
cbar = fig.colorbar(sm)
plt.axis('off')
plt.show()

Create this normalised IMD value before everything is converted to json:

In [ ]:
merged['IMDNorm']=merged['IMD']/(merged['IMD'].max())

Read into json and return a str

In [ ]:
import json
merged_json = json.loads(merged.to_json())
json_data = json.dumps(merged_json)
assert isinstance(json_data,str)

### Interactive plots with Bokeh

Imports!

In [ ]:
from bokeh.io import show, output_file, output_notebook, save
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
import bokeh.palettes
from bokeh.models.tickers import FixedTicker

In [ ]:
geosource = GeoJSONDataSource(geojson = json_data)
# use viridis colour map as before
palette = bokeh.palettes.viridis(8)
palette = palette[::-1] # runs the colour palette backwards, personal preference

color_mapper = LinearColorMapper(palette = palette, low = 0, high = 4000)
cbarTicker = FixedTicker(ticks=np.arange(0,4000,500))

#Add a hover tooltip to the map
hover = HoverTool(tooltips = [ ('Area','@Area'),('IMD', '@IMD'),('Cases', '@Cases'),('Date','@Date')])

#Create a colourbar with the viridis palette we set above
color_bar = ColorBar(color_mapper=color_mapper, ticker=cbarTicker,label_standoff=8,width = 400, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')

map_plot = figure(title = 'Lab-Confirmed COVID-19 Cases By Area', plot_height = 600 , plot_width = 400, toolbar_location = None,tools=[hover])
# turn off gridlines
map_plot.xgrid.grid_line_color = None
map_plot.ygrid.grid_line_color = None

map_plot.patches('xs','ys', source = geosource,fill_color = {'field' :'Cases', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.1, fill_alpha = .8)
map_plot.add_layout(color_bar, 'below')

# this line is required to display bokeh plots inline in jupyter notebooks:
output_notebook()

# draw the plot
show(map_plot)


In [ ]:
geosource = GeoJSONDataSource(geojson = json_data)
# use viridis colour map as before
palette = bokeh.palettes.viridis(10)
palette = palette[::-1]

color_mapper = LinearColorMapper(palette = palette, low = 0, high = 50)
cbarTicker = FixedTicker(ticks=np.arange(0,50,5))

#Add a hover tooltip to the map
hover = HoverTool(tooltips = [ ('Area','@Area'),('IMD', '@IMD'),('Cases', '@Cases'),('Date','@Date')])

#Create a colourbar with the viridis palette we set above
color_bar = ColorBar(color_mapper=color_mapper, ticker=cbarTicker,label_standoff=8,width = 400, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')

map_plot = figure(title = 'Average Index of Multiple Deprivation By Area', plot_height = 600 , plot_width = 400, toolbar_location = None,tools=[hover])
# turn off gridlines
map_plot.xgrid.grid_line_color = None
map_plot.ygrid.grid_line_color = None

map_plot.patches('xs','ys', source = geosource,fill_color = {'field' :'IMD', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.1, fill_alpha = .8)
map_plot.add_layout(color_bar, 'below')

# this line is required to display bokeh plots inline in jupyter notebooks:
output_notebook()

# draw the plot
show(map_plot)

In [ ]:
geosource = GeoJSONDataSource(geojson = json_data)
# use viridis colour map as before
palette = bokeh.palettes.viridis(11)
palette = palette[::-1]

color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.1)
cbarTicker = FixedTicker(ticks=np.arange(0,1.1,0.1))

#Add a hover tooltip to the map
hover = HoverTool(tooltips = [ ('Area','@Area'),('IMD', '@IMDNorm'),('Cases', '@Cases'),('Date','@Date')])

#Create a colourbar with the viridis palette we set above
color_bar = ColorBar(color_mapper=color_mapper, ticker=cbarTicker,label_standoff=8,width = 400, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')

map_plot = figure(title = 'Normalised Average Index of Multiple Deprivation By Area', plot_height = 600 , plot_width = 400, toolbar_location = None,tools=[hover])
# turn off gridlines
map_plot.xgrid.grid_line_color = None
map_plot.ygrid.grid_line_color = None

map_plot.patches('xs','ys', source = geosource,fill_color = {'field' :'IMDNorm', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.1, fill_alpha = .8)
map_plot.add_layout(color_bar, 'below')

# this line is required to display bokeh plots inline in jupyter notebooks:
output_notebook()

# draw the plot
show(map_plot)

## Extra datasets - Work in progress

Death statistics per lower tier local authority (March-May 2020)

In [ ]:
import openpyxl
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
depURL="https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fdeathsinvolvingcovid19bylocalareaanddeprivation%2f1march2020to31may2020/referencetablesworkbook1.xlsx"
depRequest=requests.get(depURL)
output=open('../data/covid_deprivation_gender.xlsx','wb')
output.write(depRequest.content)
output.close()

In [ ]:
death_area=pd.read_excel('../data/covid_deprivation_gender.xlsx',"Table 2")
death_area=death_area.drop([0,1])
death_area.index=np.arange(0,len(death_area))
death_area=death_area.head(3486)
new_header = death_area.iloc[0] #grab the first row for the header
death_area = death_area[1:] #take the data less the header row
death_area.columns = new_header #set the header row as the df header
death_area.index=np.arange(0,len(death_area))

In [ ]:
death_area=death_area.dropna(how='all')
death_area=death_area.dropna(axis=1,how='all')
death_area.columns=['Cause',
 'Sex',
 'Geography type',
 'Area code',
 'Area name',
 'March',
 'marchnan',
 'marchnan2',
 'marchnan3',
 'marchnan4',
 'April',
 'aprilnan',
 'aprilnan2',
 'aprilnan3',
 'aprilnan4',
 'May',
 'maynan',
 'maynan2',
 'maynan3',
 'maynan4',
 'MarchMayDeaths',
 'MarchMayRate',
 'MarchMayNaN2',
 'MarchMayRateLowCI',
 'MarchMayRateHighCI']
death_area=death_area[['Cause','Sex','Geography type','Area code','Area name','MarchMayDeaths','MarchMayRate','MarchMayRateLowCI','MarchMayRateHighCI']]

In [ ]:
death_area.columns=['Cause',
 'Sex',
 'GeoType',
 'Area code',
 'Area name',
 'Deaths',
 'Rate',
 'RateLowCI',
 'RateHighCI']

In [ ]:
death_area=death_area.drop(index=[0])
death_area=death_area[death_area['Cause'].str.match('COVID-19')]
death_area.to_csv("../data/mortality_stats_byArea_EnglandWales_MarchMay2020.csv")